In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Load Data
df = pd.read_excel("./insights/ogsup.xlsx")  # Replace with actual file path

# Select features and target
features = ['temperature', 'windspeed', 'humidity', 'precipitation', 'dewpoint',
            'cloud_cover', 'pressure', 'solar_radiation', 'sunshine_duration', 
            'ndvi', 'elevation']
target = 'breed'

# Normalize features
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

# Convert timestamp to datetime and sort by individual and time
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values(by=['individual-local-identifier', 'timestamp'])

# Convert data into sequences for LSTM
sequence_length = 24 * 7  # 7 days of hourly data

def create_sequences(data, target, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length][features].values)
        y.append(data.iloc[i+seq_length][target])
    return np.array(X), np.array(y)

X, y = create_sequences(df, target, sequence_length)

# Train-test split (shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

# Print class distribution
print("Train set class distribution:", np.bincount(y_train))
print("Test set class distribution:", np.bincount(y_test))

# Compute class weights to handle imbalance
class_weights = compute_class_weight("balanced", classes=np.unique(y_train), y=y_train)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Build Improved LSTM Model
model = Sequential([
    Input(shape=(sequence_length, len(features))),  # Define input shape explicitly
    
    LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    
    LSTM(32, return_sequences=False, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(16, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early Stopping with higher patience
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train model with class weights and early stopping
model.fit(X_train, y_train, epochs=100, batch_size=64,  # Changed batch size to 64
          validation_data=(X_test, y_test), class_weight=class_weight_dict,
          callbacks=[early_stopping])


# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# Predict
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, zero_division=1))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_classes)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=[0,1], yticklabels=[0,1])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

import pickle
model.save("lstm_hornbill_model.keras")  # Save in the new Keras format

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)


In [13]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import joblib

def predict_new_locations(model, scaler, new_data_path, sequence_length=24*7, features=None, feature_weights=None):
    """
    Test the trained LSTM model on data from new locations with weighted features.
    
    Parameters:
    -----------
    model : keras.Model
        Trained LSTM model
    scaler : StandardScaler
        Fitted scaler used during training
    new_data_path : str
        Path to the new data file
    sequence_length : int
        Length of sequences used during training
    features : list
        List of feature names used during training
    feature_weights : dict
        Dictionary mapping feature names to importance weights
    
    Returns:
    --------
    DataFrame with original data and predictions
    """
    if features is None:
        features = ['temperature', 'windspeed', 'humidity', 'precipitation', 
                    'dewpoint', 'cloud_cover', 'pressure', 'solar_radiation', 
                    'sunshine_duration', 'ndvi', 'elevation']
    
    if feature_weights is None:
        feature_weights = {
            'temperature': 3.0, 'humidity': 3.0
        }
    
    # Load new data
    try:
        new_df = pd.read_excel(new_data_path)
        # new_df=new_df[1:500]
        # new_df['latitude']=new_df['location-lat'].round(2)
        # new_df['longitude']=new_df['location-long'].round(2)
    except Exception as e:
        print(f"Error loading data: {e}")
        return None
    
    # Check if required columns exist
    required_columns = ['latitude', 'longitude', 'timestamp'] + features
    missing_columns = [col for col in required_columns if col not in new_df.columns]
    if missing_columns:
        print(f"Missing required columns: {missing_columns}")
        return None
    
    # Convert timestamps to datetime and sort data
    new_df['timestamp'] = pd.to_datetime(new_df['timestamp'])
    
    # Apply feature weights before normalization
    for feature in features:
        new_df[feature] *= feature_weights.get(feature, 1.0)  # Default weight is 1.0 if not found
    
    # Normalize features using the same scaler as in training
    new_df[features] = scaler.transform(new_df[features])
    
    # Create sequences for prediction, grouped by location (latitude, longitude)
    X_new, sequence_info = [], []
    
    for (lat, lon), group in new_df.groupby(['latitude', 'longitude']):
        group = group.sort_values(by='timestamp')
        
        if len(group) >= sequence_length:
            for i in range(len(group) - sequence_length + 1):
                sequence = group.iloc[i:i+sequence_length][features].values
                X_new.append(sequence)
                
                # Store metadata about this sequence
                sequence_info.append({
                    'latitude': lat,
                    'longitude': lon,
                    'start_time': group.iloc[i]['timestamp'],
                    'end_time': group.iloc[i+sequence_length-1]['timestamp']
                })
    
    if not X_new:
        print("No valid sequences could be created from the new data")
        return None
    
    X_new = np.array(X_new)
    
    # Make predictions
    predictions = model.predict(X_new)
    predictions_binary = (predictions > 0.5).astype(int)
    
    # Create results DataFrame
    results_df = pd.DataFrame(sequence_info)
    results_df['predicted_probability'] = predictions.flatten()
    results_df['predicted_class'] = predictions_binary.flatten()
    
    # Add confidence levels
    results_df['confidence'] = np.where(
        predictions.flatten() > 0.5,
        predictions.flatten(),
        1 - predictions.flatten()
    )
    
    # Add interpretation
    def get_confidence_level(conf):
        if conf >= 0.9: return "Very High"
        elif conf >= 0.75: return "High"
        elif conf >= 0.6: return "Moderate"
        else: return "Low"
    
    results_df['confidence_level'] = results_df['confidence'].apply(get_confidence_level)
    
    return results_df

# Example usage:
if __name__ == "__main__":
    try:
        # Load trained model and scaler
        model = load_model('lstm_hornbill_model.keras')  # Replace with actual model path
        scaler = joblib.load('scaler.pkl')  # Replace with actual scaler path
        
        # Test on new location data
        new_data_path = "ogtest1.xlsx"  # Replace with actual test file

        results = predict_new_locations(model, scaler, new_data_path)
        
        if results is not None:
            # Display summary statistics
            print("\nPrediction Summary:")
            print("-" * 50)
            print(f"Total sequences analyzed: {len(results)}")
            print("\nPredictions by class:")
            print(results['predicted_class'].value_counts())
            print("\nConfidence level distribution:")
            print(results['confidence_level'].value_counts())
            
            # Save results to Excel
            results.to_excel("prediction_results.xlsx", index=False)
            print("\nResults saved to 'prediction_results.xlsx'")
            
            # Optional: Display high-confidence predictions
            print("\nHigh confidence predictions (confidence >= 0.9):")
            high_conf = results[results['confidence'] >= 0.9]
            print(high_conf[['latitude', 'longitude', 'predicted_class', 'confidence']].head())
            
    except Exception as e:
        print(f"Error: {e}")

c:\Users\visha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 16 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step

Prediction Summary:
--------------------------------------------------
Total sequences analyzed: 5

Predictions by class:
predicted_class
0    3
1    2
Name: count, dtype: int64

Confidence level distribution:
confidence_level
Very High    5
Name: count, dtype: int64

Results saved to 'prediction_results.xlsx'

High confidence predictions (confidence >= 0.9):
   latitude  longitude  predicted_class  confidence
0   -4.0000    31.1667                0    0.915105
1    5.5333    23.3000                0    0.999020
2   10.5167    26.5833                1    0.999408
3   16.0000    20.6667                1    0.999819
4   28.5000    28.5000                0    0.999919
